In [21]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv('data/data_table/data_sheet1.csv')
df = df.rename(columns={'Unnamed: 0': 'ranking'})
df = df[df.columns[:25]]

In [40]:
df.columns


Index(['ranking', 'each_original_data', 'ion1', 'loss1', 'loss_sign1',
       'charge1', 'mass1', 'ion2', 'loss2', 'loss_sign2', 'charge2', 'mass2',
       'addition_mass1', 'addition_mass2', 'ion_mass1', 'ion_mass2',
       'correct_mass1', 'correct_mass2', 'mass_difference1',
       'mass_difference2', 'm1+m2', '2m1+m2', 'm1+2m2', 'chosen_sum_from',
       'chosen_sum'],
      dtype='object')

In [94]:
def create_mass_conserve_line(row):
    ion1,ion2 = row['ion1'], row['ion2']
    loss1,loss2 = row['loss1'], row['loss2']
    if pd.isna(loss1):
        loss1 = ''
    if pd.isna(loss2):
        loss2 = ''
        
        
    #print((type(ion1) == str) and (ion1[0] == 'a' and ion1[:2] != 'ai'))
    #print((type(ion2) == str) and (ion2[0] == 'a' and ion2[:2] != 'ai')and (loss1 == '' and loss2 == ''))
    if (type(ion1) == str) and (ion1[0] == 'a' and ion1[:2] != 'ai') and (loss1 == '' and loss2 == ''):
        combined_loss = 'a'
    elif (type(ion2) == str) and (ion2[0] == 'a' and ion2[:2] != 'ai') and (loss1 == '' and loss2 == ''):
        combined_loss = 'a'
        
    elif loss1 != '' and loss2 != '':
        combined_loss = loss1 + ' + ' +loss2

    else:
        combined_loss = loss1 + loss2
    if combined_loss == '':
        combined_loss = 'Parent'
    return combined_loss


In [98]:
df['conserve_line'] = df.apply(create_mass_conserve_line, axis=1)

In [143]:
def peptie_arrange(length):
    result = []
    for i in range(1, length):
        result.append(f'b{i}y{length - i}')
    return result
        

In [179]:
rows = ['Parent','NH3','H2O', 'NH3 + H2O','H2O + NH3', 'a']
columns = peptie_arrange(14)
df['loss1_m'] = df['loss1'].fillna('')
df['loss2_m'] = df['loss2'].fillna('')

In [180]:
reuslt_df = pd.DataFrame(index=rows, columns=columns)

In [185]:
for index, each_row in df.iterrows():
    the_column = -1
    if (type(each_row['ion1']) == str and type(each_row['ion2']) == str):
        the_column = [each_row['ion1'], each_row['ion2']]
        the_column.sort()
        the_column = the_column[0] + the_column[1]
        the_column = the_column.replace('a', 'b')
    if (each_row['conserve_line'] in reuslt_df.index) and (the_column in reuslt_df.columns):
        print(each_row['conserve_line'])
        reuslt_df.at[each_row['conserve_line'], the_column] = each_row['ion1'] + each_row['loss1_m'] + each_row['ion2']+ each_row['loss2_m'] + ' \n ' +\
        each_row['chosen_sum_from'] + ' \n ' + str(round(each_row['mass_difference1'] + each_row['mass_difference2'], 2))
reuslt_df = reuslt_df.fillna('--')

Parent
Parent
Parent
NH3
NH3
NH3
NH3
NH3
a
NH3
H2O + NH3
Parent


In [186]:
reuslt_df

,b1y13,b2y12,b3y11,b4y10,b5y9,b6y8,b7y7,b8y6,b9y5,b10y4,b11y3,b12y2,b13y1
Parent,--,--,--,--,--,--,y7b7 \n m1+m2 \n 0.11,y6b8 \n m1+m2 \n 0.18,y5b9 \n m1+m2 \n -0.01,--,y3b11 \n m1+m2 \n -0.21,--,--
NH3,--,--,--,--,--,--,b7y7NH3 \n m1+m2 \n 0.87,y6NH3b8 \n m1+m2 \n 0.03,y5NH3b9 \n m1+m2 \n -0.02,y4NH3b10 \n m1+m2 \n -0.37,y3NH3b11 \n m1+m2 \n 0.31,--,--
H2O,--,--,--,--,--,--,--,--,--,--,--,--,--
NH3 + H2O,--,--,--,--,--,--,--,--,--,--,--,--,--
H2O + NH3,--,--,--,--,--,--,y7H2Ob7NH3 \n m1+m2 \n 0.18,--,--,--,--,--,--
a,--,--,--,--,--,--,--,y6a8 \n m1+m2 \n 0.4,--,--,--,--,--


In [163]:
df.columns

Index(['ranking', 'each_original_data', 'ion1', 'loss1', 'loss_sign1',
       'charge1', 'mass1', 'ion2', 'loss2', 'loss_sign2', 'charge2', 'mass2',
       'addition_mass1', 'addition_mass2', 'ion_mass1', 'ion_mass2',
       'correct_mass1', 'correct_mass2', 'mass_difference1',
       'mass_difference2', 'm1+m2', '2m1+m2', 'm1+2m2', 'chosen_sum_from',
       'chosen_sum', 'conserve_line'],
      dtype='object')

In [190]:
import pandas as pd
from html2image import Html2Image

# 1. Your original DataFrame and color dictionary
data = {
    'b12y1 | b2y1': ['(no loss,no loss) 0.31', '(no loss,NH3) -0.25', '(no loss,H2O)', '--'],
    'b3y10 | b4y9 | b5y8': ['--', '(NH3,no loss) -0.17', '--', '--'],
}
df = pd.DataFrame(data, index=['Parent', 'NH3', 'H2O', 'a'])

color_dict = {
    'NH3': 'blue',
    'H2O': 'red',
}

# 2. Create a copy of the DataFrame to modify its content
# It's good practice to work on a copy
df_display = df.copy().astype(str)

# 3. Iterate through the rows and inject HTML directly into the data
for index_name in df_display.index:
    if index_name in color_dict:
        color = color_dict[index_name]
        substring_to_color = index_name
        
        # Create the replacement HTML string
        replacement_html = f"<span style='color: {color}; font-weight: bold;'>{substring_to_color}</span>"
        
        # Modify the row in our display DataFrame
        # We use regex=False to ensure it treats the string literally
        df_display.loc[index_name] = df_display.loc[index_name].str.replace(
            substring_to_color, replacement_html, regex=False
        )

# 4. Now, apply simple, table-wide styles. No complex .apply() is needed.
styled_df = df_display.style.set_properties(**{
    'border': '1px solid black',
    'text-align': 'center'
})

# 5. Export to HTML (with escape=False) and then to PNG
html_output = styled_df.to_html(escape=False)
with open("colored_substrings.html", "w") as f:
    f.write(html_output)

hti = Html2Image()
hti.screenshot(html_file='colored_substrings.html', save_as='colored_substrings.png')

22134 bytes written to file /Users/kevinmbp/Desktop/2D_spec_dict/colored_substrings.png


['/Users/kevinmbp/Desktop/2D_spec_dict/colored_substrings.png']

In [192]:
import pandas as pd
from html2image import Html2Image

# Your setup code remains the same
data = {
    'b12y1 | b2y1': ['(no loss,no loss) 0.31', '(no loss,NH3) -0.25', '(no loss,H2O)', '--'],
    'b3y10 | b4y9 | b5y8': ['--', '(NH3,no loss) -0.17', '--', '--'],
}
df = pd.DataFrame(data, index=['Parent', 'NH3', 'H2O', 'a'])
color_dict = {
    'NH3': 'blue',
    'H2O': 'red',
    'Parent': 'green'
}
df_display = df.copy().astype(str)
for index_name in df_display.index:
    if index_name in color_dict:
        color = color_dict[index_name]
        replacement_html = f"<span style='color: {color}; font-weight: bold;'>{index_name}</span>"
        df_display.loc[index_name] = df_display.loc[index_name].str.replace(
            index_name, replacement_html, regex=False
        )

# 💡 THE FIX: Rewrite the function to handle a Series
def color_index_series(index_labels_series):
    """
    Takes a Series of index labels and returns a Series of CSS styles.
    """
    # Use .map() to apply the logic to each label in the Series
    return index_labels_series.map(
        lambda label: f"color: {color_dict.get(label, 'black')}; font-weight: bold;"
    )

# Now, call the new function
styled_df = df_display.style.apply_index(color_index_series, axis="index") \
                          .set_properties(**{
                              'border': '1px solid black',
                              'text-align': 'center'
                          })

# Exporting remains the same
html_output = styled_df.to_html(escape=False)
with open("colored_table_and_index.html", "w") as f:
    f.write(html_output)

hti = Html2Image()
hti.screenshot(html_file='colored_table_and_index.html', save_as='colored_table_and_index.png')

22220 bytes written to file /Users/kevinmbp/Desktop/2D_spec_dict/colored_table_and_index.png


['/Users/kevinmbp/Desktop/2D_spec_dict/colored_table_and_index.png']